In [1]:
include("../library/utils.jl")
include("../library/equation.jl")
include("../library/integraton.jl")
using .Utils: truncate_to_decimal_places
using Plots

# Q1:

**Solve exp(−x) − x = 0 using fixed-point method, accurate up to 4 places in decimal.**

$$
e^{-x} -x = 0
$$

Solving the given equation is equivalent to solving this equation:
$$
x = e^{-x}
$$

according to the fixed point method, let our function $\phi(x)$ be:

$$
\phi(x) = e^{-x}
$$

**Note:** The used `Eqn.fixed_point_iteration` function can be found in the [equation.jl](../library/equation.jl) file.

In [2]:
function phi(x)
    return exp(-x)
end

ans = Eqn.fixed_point_iteration(phi, 1)
print("Fixed point iteration: ")
println(truncate_to_decimal_places(ans, 4))

Fixed point iteration: 0.5671


# Q2

Use Simpson’s rule and appropriate Gaussian quadrature to evaluate the following integral accurate up to 6 places in decimal

$$
\int_0^1 \sqrt{1+x^4} dx
$$

In [9]:
function f(x)
	return sqrt(1 + x^4)
end

simp = Integration.simpson_rule(f, 0, 1, 4)
print("Simpson's rule: ")
println(truncate_to_decimal_places(simp, 6))

gquad = Integration.gaussian_quadrature(f, 0, 1, 4)
print("Gaussian quadrature: ")
println(truncate_to_decimal_places(gquad, 6))
# print()

Simpson's rule: 1.089429
Gaussian quadrature: 1.089423


# Q3
Solve the following ODE with RK4 with interval sizes 0.5, 0.2, 0.05 and 0.01

$$
\frac{dy}{dx} = \frac{5x^2 - y}{e^{x+y}}
$$

where $y(0) = 1$

Tabulate the results.

In [11]:
function f(x, y)
	return (5*x^2 - y) / exp(x + y)
end



f (generic function with 2 methods)